In [ ]:
# !pip install -r ../requirements.txt
import os
import sys
from pathlib import Path

# Ensure repo root is on sys.path when running from notebooks/
repo_root = Path.cwd().resolve().parent
if repo_root.name == 'notebooks':
    repo_root = repo_root.parent
sys.path.append(str(repo_root))

import pandas as pd
# !pip install -r ../requirements.txt
import os
import sys
from pathlib import Path

# Ensure repo root is on sys.path when running from notebooks/
repo_root = Path.cwd().resolve().parent
if repo_root.name == 'notebooks':
    repo_root = repo_root.parent
sys.path.append(str(repo_root))

import pandas as pd
from src.data_loader import load_news_csv, load_price_csv
from src.sentiment import score_headlines_df
from src.correlation import compute_daily_returns, aggregate_daily_sentiment, merge_sentiment_returns, pearson_correlation

news_path = 'C:/Users/alexo/Desktop/File/10Academy/week1/Data/newsData/raw_analyst_ratings.csv'
price_path = 'C:/Users/alexo/Desktop/File/10Academy/week1/Data/yfinance_data/Data/AAPL.csv'

try:
    news = load_news_csv(news_path)
    news = score_headlines_df(news)
    price = load_price_csv(price_path)
    returns = compute_daily_returns(price)
    sent_agg = aggregate_daily_sentiment(news)
    merged = merge_sentiment_returns(sent_agg, returns, left_on='date', right_on='Date')
    corr = pearson_correlation(merged)
    print('Pearson correlation (avg_sentiment, daily_return):', corr)
except Exception as e:
    print('Run with sample datasets in ../data to reproduce the analysis.')
    print('Week-1 brief path:', '../10 Academy - AIM - Week 1.docx')
from src.sentiment import score_headlines_df
from src.correlation import compute_daily_returns, aggregate_daily_sentiment, merge_sentiment_returns, pearson_correlation

news_path = 'C:/Users/alexo/Desktop/File/10Academy/week1/Data/newsData/raw_analyst_ratings.csv'
price_path = 'C:/Users/alexo/Desktop/File/10Academy/week1/Data/yfinance_data/Data/AAPL.csv'

try:
    news = load_news_csv(news_path)
    news = score_headlines_df(news)
    price = load_price_csv(price_path)
    returns = compute_daily_returns(price)
    sent_agg = aggregate_daily_sentiment(news)
    merged = merge_sentiment_returns(sent_agg, returns, left_on='date', right_on='Date')
    corr = pearson_correlation(merged)
    print('Pearson correlation (avg_sentiment, daily_return):', corr)
except Exception as e:
    print('Run with sample datasets in ../data to reproduce the analysis.')
    print('Week-1 brief path:', '../10 Academy - AIM - Week 1.docx')
